In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/run/media/alf/datos/misrepositorios/docencia/analisis-practicas-julia"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
using SymPy  # Para el cálculo simbólico.
using Plots  # Para el dibujo de gráficas.
using Makie, GLMakie # Para el dibujo de gráficas en 3d.
using HCubature # Para el cálculo numérico de integrales múltiples.
using LaTeXStrings  # Para usar código LaTeX en los gráficos.
using LinearAlgebra # Para el módulo y el producto escalar de vectores.

In [3]:
using HCubature
f(x, y) = 10
f(v) = f(v...)
hcubature(f, [0,0], [1,5])

(50.0, 7.105427357601002e-15)

In [4]:
f(x, y) = 10x
f(v) = f(v...)
hcubature(f, [0,0], [1,5])

(25.0, 3.552713678800501e-15)

In [5]:
f(x, y) = x^2+y^2<1 ? 10 : 0
f(v) = f(v...)
#hcubature(f, [-1,-1], [1,1])

f (generic function with 3 methods)

In [6]:
f(x, y) = x^2+y^2<1 ? sqrt(1-x^2-y^2) : 0
f(v) = f(v...)
hcubature(f, [-1,-1], [1,1])

(2.0943951101375893, 3.12087277156216e-8)

In [7]:
using SymPy
@syms x y
f(x, y) = x + y
integrate(f(x,y), (y, 0, 2), (x, 0, 1))

3

In [8]:
integrate(f(x,y), (x, 0, 1), (y, 0, 2))

3

In [9]:
@syms z
f(x, y, z) = x * y * z
integrate(f(x,y,z), (z, 0, 3), (y, 0, 2), (x, 0, 1))

9/2

In [10]:
@syms z
f(x, y, z) = x * y * z
integrate(f(x,y,z), (y, 0, 2), (x, 0, 1), (z, 0, 3))

9/2

In [11]:
using SymPy
@syms x y
f(x, y) = exp(x+y)
integrate(f(x,y), (y, 0, x/2), (x, 0, 2))

              3
   2   1   2⋅ℯ 
- ℯ  + ─ + ────
       3    3  

In [12]:
integrate(f(x,y), (x, 2y, 2), (y, 0, 1))

              3
   2   1   2⋅ℯ 
- ℯ  + ─ + ────
       3    3  

In [13]:
using SymPy
@syms x y
f(x, y) = x^2*y
integrate(f(x,y), (y, x, sqrt(x)), (x, 0, 1))

1/40

In [14]:
f(x, y) = cos(x^2)
integrate(f(x,y), (x, y, 1), (y, 0, 1))

        ⎛                      ⎛√2⎞       ⎞                            
        ⎜                     C⎜──⎟⋅Γ(1/4)⎟                            
        ⎜  √2⋅sin(1)⋅Γ(1/4)    ⎝√π⎠       ⎟                 ⎛√2⎞       
  √2⋅√π⋅⎜- ──────────────── + ────────────⎟⋅Γ(1/4)   √2⋅√π⋅C⎜──⎟⋅Γ(1/4)
        ⎝    8⋅√π⋅Γ(5/4)        4⋅Γ(5/4)  ⎠                 ⎝√π⎠       
- ──────────────────────────────────────────────── + ──────────────────
                      8⋅Γ(5/4)                            8⋅Γ(5/4)     

In [15]:
integrate(f(x,y), (y, 0, x), (x, 0, 1))

sin(1)
──────
  2   